In [2]:
import json

# JSON 파일 불러오기 (경로 확인 필요)
file_path = "../데이터/highschool_english_dataset.json"

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 원본 질문 리스트 생성
questions = []
for key, value in data.items():
    if "questions" in value and "cleaned_text" in value["questions"]:
        questions.append(value["questions"]["cleaned_text"])

print(f"✅ 원본 질문 로드 완료! 총 {len(questions)}개의 질문이 있습니다.")

✅ 원본 질문 로드 완료! 총 14개의 질문이 있습니다.


# FAISS 벡터 데이터베이스(faiss_index.bin) 불러오기

In [3]:
import faiss

# FAISS 인덱스 불러오기
index = faiss.read_index("faiss_index.bin")

print(f"✅ FAISS 인덱스 불러오기 완료! 저장된 벡터 개수: {index.ntotal}")

✅ FAISS 인덱스 불러오기 완료! 저장된 벡터 개수: 14


# OpenAI 임베딩 함수 (text-embedding-ada-002)

In [4]:
import openai
import numpy as np

# OpenAI 최신 API 사용 (openai>=1.0.0)
client = openai.OpenAI()

# 임베딩 변환 함수
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding  # 최신 API 방식 적용

# FAISS에서 유사한 질문 검색

In [5]:
def search_similar_questions(query, top_k=3):
    query_vector = np.array(get_embedding(query)).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)

    # 유사 질문 리스트 생성
    similar_questions = [questions[idx] for idx in indices[0]]

    print("\n[가장 유사한 질문들]")
    for i, question in enumerate(similar_questions):
        print(f"{i+1}. {question} (유사도 거리: {distances[0][i]:.4f})")

    return similar_questions

# GPT-4와 연결하여 최종 RAG 응답 생성

In [6]:
def generate_response(query):
    # 유사한 질문 찾기
    similar_questions = search_similar_questions(query, top_k=3)

    # 검색된 질문을 프롬프트 컨텍스트로 활용
    context = "\n".join(similar_questions)

    # GPT-4 프롬프트 설정
    prompt = f"""
    You are an AI assistant. Use the following context to answer the question.

    Context:
    {context}

    Question: {query}

    Answer:
    """

    # GPT-4 API 호출 (최신 방식)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

# 예제 실행 (사용자 질문 입력)

In [7]:
# 예제 실행
user_query = "How can we prevent water scarcity?"
answer = generate_response(user_query)

print("\n[GPT-4의 답변]\n", answer)


[가장 유사한 질문들]
1. 영 제 3 교시 [1~3] 밑줄 친 부분의 뜻으로 가장 적절한 것을 고르시오. 1. David put a lot of effort into the exam. ① 노력 ② 의미 ③ 조언 ④ 좌절 2. Many countries suffer from a lack of water. ① 절약하다 ② 이용하다 ③ 낭비하다 ④ 고통받다 3. A: Excuse me, can I have soup instead of salad? B: Sure. I will bring it in a minute. ① ～로써 ② ～대신에 ③ ～덕분에 ④ ～와 함께 4. 두 단어의 의미 관계가 나머지 셋과 다른 것은? ① joy－sadness ② furniture－sofa ③ subject－science ④ animal－chimpanzee 5. 다음 광고문에서 언급되지 않은 것은? Moonlight Tour of the Beautiful Palace! Opening: Tuesday to Friday 7 p.m.－9 p.m. Reservation: Book on our website one day before (www.moonlight.co.kr) Entrance Fee: Free for everyone ① 운영 시간 ② 예약 방법 ③ 준비물 ④ 입장료 [6~8] 빈칸에 공통으로 들어갈 말로 가장 적절한 것을 고르시오. 6. ◦Can you explain to use the copy machine? ◦I can’t understand he solved the problem. ① what ② that ③ who ④ how 7. ◦The flood had caused serious to the village. ◦Don’t the surface of the table with the pencil. ① damage ② gesture ③ comment ④ concern 8. ◦All the people focused her performance. ◦This island’

In [8]:
# ✅ FAISS에서 랜덤 질문 검색 (RAG 적용)
def get_random_question_from_faiss():
    random_idx = random.randint(0, index.ntotal - 1)

    # FAISS에서 랜덤 벡터를 검색하여 가장 유사한 질문 찾기
    query_vector = np.zeros((1, 1536), dtype="float32")  # 빈 벡터로 검색 (랜덤 선택을 위해)
    distances, indices = index.search(query_vector, 1)

    retrieved_index = indices[0][0]
    if retrieved_index >= 0:
        similar_question = f"🔍 FAISS에서 검색된 질문 (인덱스 {retrieved_index})"
    else:
        similar_question = "❌ 적절한 질문을 찾을 수 없습니다."

    return similar_question

In [9]:
def get_random_question_from_faiss():
    # FAISS에서 임의의 질문 검색
    question_index = np.random.randint(0, index.ntotal)
    retrieved_question = index.reconstruct(question_index)

    print("🔍 FAISS 검색 결과:", retrieved_question)

    # 데이터가 예상한 형식인지 확인
    if not isinstance(retrieved_question, str):
        return None  # 잘못된 데이터 형식이면 None 반환

    # 문제를 적절히 분리 (예: 문제 | 선택지 | 정답 형식일 경우)
    parts = retrieved_question.split("|")
    
    if len(parts) == 3:
        question = parts[0].strip()
        options = parts[1].strip().split(",")  # 선택지가 쉼표로 구분되어 있다고 가정
        correct_answer = parts[2].strip()
        
        return question, options, correct_answer
    else:
        return None  # 형식이 올바르지 않으면 None 반환

In [11]:
def get_random_question_from_faiss():
    try:
        if index.ntotal == 0:
            print("⚠️ FAISS 데이터베이스가 비어 있습니다.")
            return None

        # 무작위 인덱스 선택
        random_idx = random.randint(0, index.ntotal - 1)

        # 벡터 검색 수행
        distances, indices = index.search(np.random.rand(1, index.d).astype('float32'), 1)

        # 반환값 확인
        if indices[0][0] == -1:
            print("⚠️ 검색된 질문이 없습니다.")
            return None

        # 검색된 질문 가져오기
        retrieved_question = questions[indices[0][0]]  # 이 부분에서 `questions` 리스트 필요

        # 예제 선택지 및 정답 추가 (더미 데이터)
        options = ["A", "B", "C", "D"]
        correct_answer = random.choice(options)

        return retrieved_question, options, correct_answer
    except Exception as e:
        print(f"❌ FAISS 검색 오류: {e}")
        return None